In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 3)

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
df_order = pd.read_sql(sql, conlite)
df_order

,id,trade,name,qty,price,active,reason,market
0,689,B,BGC,9000,10.4,0,DOS,SET


In [4]:
names = df_order['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conlite)
df_stocks


SELECT * FROM stocks WHERE name = 'BGC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,689,BGC,13.1,9.2,O,10.4,0,34.52,0.7,0,9000,-4,4,0,0,0,DOS,SET


In [13]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name spd reason qty target current percent active'.split()

### Create orders from stocks after adjust stocks in port_lite from consensus

In [6]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

23

In [7]:
sql = '''
INSERT INTO orders
SELECT id, 'B', name, qty, buy_target, 0, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
rp = conlite.execute(sql)
rp.rowcount

16

In [8]:
sql = '''
INSERT INTO orders
SELECT id,'S',name, qty, sell_target, 0, reason, market
FROM stocks
WHERE status IN ('I','S')'''
rp = conlite.execute(sql)
rp.rowcount

7

In [9]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_orders.shape[0]

23

In [10]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=False, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=False, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=False, index=False)

### After call Ord-log.bat

In [11]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [16]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9,11])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty','col6': 'target',
                     'col7': 'current', 'col8': 'chg', 'col9': 'percent', 'col11': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)